# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [30]:
DATA_FOLDER = 'Data' # Use the data folder provided in Tutorial 02 - Intro to Pandas.

In [6]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn', Mutes warnings when copying a slice from a DataFrame.

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('notebook')

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average per month* of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

In [7]:
from datetime import datetime
from datetime import date, time
import glob

In [8]:
def treat_data(df,c):
    # Standardize data descriptions
    df=df.rename(columns = {'date':'Date'})
    df=df.rename(columns = {'variable':'Description'})
    df=df.rename(columns = {'Variable':'Description'})
    df=df.rename(columns={'Unnamed: 18':'Unnamed_18_'+c})
    df=df.replace(['New case/s (confirmed)','New cases of confirmed','new_confirmed'],'new cases')
    df=df.replace(['etc_new_deaths','Newly reported deaths','New deaths registered today (confirmed)',
                  'New deaths registered'],'new deaths')
    df=df.loc[df['Description'].isnull()==False]
    
    # Drop unwanted values
    df=df.dropna(how='all')
    df=df.replace(['-','Nr','NR'],np.nan)
    
    # transform dates to datetime
    df['Date'] = df['Date'].astype(str)
    df = df.loc[df['Date'].apply(lambda x: len(x)<=10)]
    df['Date'] = pd.to_datetime(df['Date'])
    
    # set desired indexes
    df = df.set_index(['Date','Description'])
    s = pd.Series([c], index=df.columns) #add an index for the country id
    df = df.transpose()
    df.set_index(s, append=True, inplace=True)
    df.index.names = ['Location','Country']
    
    return df

In [9]:
# Import Data
DATA_FOLDER='Data'
countries=pd.Series(['guinea_data','liberia_data','sl_data'],['Guinea','Liberia','Sierra Leone'])
name = 0
frame = pd.DataFrame()
list_ = []
d={}

for c in range(3):
    list_=[]
    print('Compute data extraction for '+countries.index[c])
    
    path=DATA_FOLDER+'/ebola/'+countries[c]
    allFiles = glob.glob(path + "/*.csv")
    
    for file_ in allFiles:
        df = pd.read_csv(file_,index_col=None, header=0)
        list_.append(df)
    frame=pd.concat(list_)
    frame=treat_data(frame,countries.index[c]) #import Data and keep country info
    name+=1
    d[c]=frame
    
# concat contries and display
dfs = [d[0], d[1], d[2]]
df = pd.concat(dfs,axis=1)
df = df.sort_index(level=1,axis=0)
df = df.swaplevel()
df.fillna('unknown')

Compute data extraction for Guinea
Compute data extraction for Liberia
Compute data extraction for Sierra Leone


Date                                            2014-08-31  \
Description                          New cases of suspects   
Country      Location                                        
Guinea       Beyla                                 unknown   
             Boffa                                 unknown   
             Conakry                                     1   
             Coyah                                 unknown   
             Dabola                                unknown   
             Dalaba                                unknown   
             Dinguiraye                            unknown   
             Dubreka                               unknown   
             Forecariah                            unknown   
             Gueckedou                                   2   
             Kerouane                              unknown   
             Kindia                                unknown   
             Kissidougou                           unknown   
             Kouroussa                             unknown   
             Lola                                  unknown   
             Macenta                                     6   
             Mzerekore                             unknown   
             Nzerekore                             unknown   
             Pita                                  unknown   
             Siguiri                               unknown   
             Telimele                              unknown   
             Totals                                      9   
             Yomou                                 unknown   
Liberia      Bomi County                           unknown   
             Bong County                           unknown   
             Gbarpolu County                       unknown   
             Grand Bassa                           unknown   
             Grand Cape Mount                      unknown   
             Grand Gedeh                           unknown   
             Grand Kru                             unknown   
...                                                    ...   
             Nimba County                          unknown   
             River Gee County                      unknown   
             RiverCess County                      unknown   
             Sinoe County                          unknown   
             Unnamed_18_Liberia                    unknown   
Sierra Leone 34 Military Hospital                  unknown   
             Bo                                    unknown   
             Bo EMC                                unknown   
             Bombali                               unknown   
             Bonthe                                unknown   
             Hastings-F/Town                       unknown   
             Kailahun                              unknown   
             Kambia                                unknown   
             Kenema                                unknown   
             Kenema (IFRC)                         unknown   
             Kenema (KGH)                          unknown   
             Koinadugu                             unknown   
             Kono                                  unknown   
             Moyamba                               unknown   
             National                              unknown   
             Police training School                unknown   
             Police traning School                 unknown   
             Port Loko                             unknown   
             Pujehun                               unknown   
             Tonkolili                             unknown   
             Unnamed_18_Sierra Leone               unknown   
             Western area                          unknown   
             Western area combined                 unknown   
             Western area rural                    unknown   
             Western area urban                    unknown   

Date                           

In [10]:
# Compute the daily average per month for each country of new cases and deaths

# Select data
df_average = df.loc[df.index.get_level_values('Location').isin(['Totals','National','national','totals','Total'])==False]
df_average = df_average.transpose()
df_average = df_average.loc[df_average.index.get_level_values('Description').isin(['new cases','new deaths'])]
df_average = df_average.astype(float)
df_average = df_average.transpose()
df_average = df_average.groupby(level=[0,1],axis=1).sum() # group by city

# Compute daily average
df_average = df_average.transpose() 
df_average = df_average.groupby(level=0,axis=1).sum() # group by date
df_average = df_average.groupby(by=(df_average.index.get_level_values('Date').month,df_average.index.get_level_values('Description'))).mean() #Average by month
df_average.columns.names = ['2014 Average new deaths and cases in:']
df_average.index.names = ['Month','Description']
df_average.fillna('unknown')

2014 Average new deaths and cases in:   Guinea  Liberia Sierra Leone
Month Description                                                   
6     new cases                        unknown  2.14286      unknown
      new deaths                       unknown  1.85714      unknown
7     new cases                        unknown  1.81818      unknown
      new deaths                       unknown  4.27273      unknown
8     new cases                           12.4  8.16667         19.6
      new deaths                           3.2  23.2222         5.55
9     new cases                        12.5625  6.16667      36.2759
      new deaths                           3.8  37.6087      4.66667
10    new cases                              6  1.47619      57.5357
      new deaths                            15    28.12      4.14286
11    new cases                        unknown      9.5      69.8947
      new deaths                       unknown  15.7692          1.5
12    new cases                        unknown  2173.88      54.3333
      new deaths                       unknown  unknown      unknown

## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

In [71]:
"""
1. We need to load the first MID1.xls as DataFrame and use its name as indexes and the name of the document as column
2. Load the 8 next MID@I.xls and join them with the first DataFrame (some more line indexes will be added to the DataFrame too)
3. Load the metadata from the last file, and map the column name [MID1,MID2,...,MID9] to a MultiIndex: [(MID1,EC,NAN),(MID2,NEC1,tissue),...]
4. Change all the NaN created by the join operation to the tag 'unknown'
"""

"helper function to load one excel of the name MID@i"
def getMID(i):
    return pd.read_excel(DATA_FOLDER+"/microbiome/MID"+str(i)+".xls",index_col=0, sheetname='Sheet 1', header=None, names = ['MID'+str(i)])

"load all the MID.xls and join them together"
a = getMID(1)
for i in range(2,10):
    m = getMID(i)
    a = a.join(m, how='outer')
    
"split the full name of the bacterie into multiIndex"
#a.index = pd.MultiIndex.from_tuples([tuple(k.split(" ")) for k,v in a.iterrows()]) 

"load the metadata from metadata.xls"
metadata = pd.read_excel(DATA_FOLDER+"/microbiome/metadata.xls",index_col=0, sheetname='Sheet1')

"create a list of tuples from each line of the metadata"
columns = list(metadata.itertuples(index=True))

"change the column name to its real tuple name in a MultiIndex"
a.columns=pd.MultiIndex.from_tuples(columns)

"check the uniqueness of the indexes"
print("all indexes are uniques =",a.index.is_unique)

"remplace NaN to unknown"
a = a.fillna('unknown')

"print the dataFrame"
a

all indexes are uniques = True


,MID1,MID2,MID3,MID4,MID5,MID6,MID7,MID8,MID9
,EXTRACTION CONTROL,NEC 1,Control 1,NEC 2,Control 2,NEC 1,Control 1,NEC 2,Control 2
,NaN,tissue,tissue,tissue,tissue,stool,stool,stool,stool
0,,,,,,,,,
"Archaea ""Crenarchaeota"" Thermoprotei Acidilobales Acidilobaceae Acidilobus",unknown,2,1,unknown,5,unknown,unknown,unknown,unknown
"Archaea ""Crenarchaeota"" Thermoprotei Acidilobales Caldisphaeraceae Caldisphaera",unknown,14,15,unknown,26,unknown,1,unknown,1
"Archaea ""Crenarchaeota"" Thermoprotei Desulfurococcales Desulfurococcaceae Ignisphaera",7,23,14,2,28,7,8,unknown,16
"Archaea ""Crenarchaeota"" Thermoprotei Desulfurococcales Desulfurococcaceae Stetteria",unknown,unknown,unknown,unknown,1,unknown,unknown,unknown,unknown
"Archaea ""Crenarchaeota"" Thermoprotei Desulfurococcales Desulfurococcaceae Sulfophobococcus",unknown,1,4,unknown,5,1,2,unknown,2
"Archaea ""Crenarchaeota"" Thermoprotei Desulfurococcales Desulfurococcaceae Thermodiscus",unknown,unknown,1,unknown,unknown,unknown,unknown,unknown,unknown
"Archaea ""Crenarchaeota"" Thermoprotei Desulfurococcales Desulfurococcaceae Thermosphaera",unknown,2,1,unknown,2,unknown,1,unknown,unknown


## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
# Write your answer here